### Description
- The objective of this agent is to provide it with a task, devide the task in steps.
- Have several agents doing each task in paralell
### Solution
- Provide a task to the LLM, ask it to provide the steps to do the task, the answer should be in json
- Parse the json dividing the several tasks.
- In paralell call the LLM providing each task, asking it to provide the solution
- Have a Validator in the end that receives each task description, solution and validate that they will work well together

### Setting up the environment 
In the future i'll be using the library utils/myopenai.py
that does the bellow and has a function that gets openai instances

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os

load_dotenv(override=True)
openai_api_key = os.getenv('GOOGLE_API_KEY')
openai_url = os.getenv('GEMINI_BASE_URL')
gemini_model = os.getenv('GEMINI_MODEL')

def get_openai_client():
    return OpenAI(base_url=openai_url, api_key=openai_api_key)

def send_message(client, message,role="user", model=gemini_model):
    return client.chat.completions.create(
        model=model,
        messages=[
            {"role": role, "content": message}
        ]
    )

def send_messages(client, messages, model=gemini_model):
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

In [ ]:
question_ai = get_openai_client()

response = send_message(question_ai, 
"I want to create a really simple Android app, This app will have 1 activity.\
The objective of this activity is to enable user to create a user or login to a existing user. \
Create tasks on how to perform this objective. I will use the tasks description to provide to another AI agent to perform the task\
So each description should also have the context of the current task, and what needs to be done \
please provide a response in json with a step number and task description, don't use markdown or any other output. THe output MUST be a valid json object")

result = response.choices[0].message.content
print(result)




In [ ]:
result = response.choices[0].message.content

results_list = json.loads(result)

tasks = results_list["tasks"]

for task in tasks:
    print(f"Task number: {task["step"]} Task Description: {task["task_description"]}")

In [ ]:
introduction = """ 
You're part of a task force to create a simple Android Activity.
The objective of this activity is to enable user to create a user or login to a existing user.
You're in charge of one of the tasks. Your task is:
"""
conclusion = """ 
Please provide the detailed steps to finish this task.
Also evaluate you work.
The response MUST be a valid json file. It must have a prperty of the description of your work, and the evaluation of you work from 0 to 100
Example: 
{{
    "description": "First add eggs, then add sugar",
    "evaluation":5
}}
"""

In [ ]:
task_answers = []

for task in tasks:
    agent = get_openai_client()
    task_response = send_message(agent, f"{introduction} {task["task_description"]} {conclusion}")
    task_answers.append(task_response.choices[0].message.content)

In [ ]:
for task_answer in task_answers:
    print(task_answer)